In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# TODO: We are using log10, discuss if we need to change the type of log that is used.


####################################
# Input Variables
####################################

# Input Variable 1 : -> IDF Threshold
idf_thrshold_no_of_failing_tcs = 15

# Input Variable 2 : -> Coverage Information
input_coverage_file_path = 'CombinedCoverageData.csv';

# Input Variable 3 : -> Y-value (from training data) Format
df_y_values = pd.read_csv('y-values.csv')

In [2]:
# IDF Threshold Calculations
idf_values = []

for x in range(1, 10):
    idf_values.append(round(x/10 * idf_thrshold_no_of_failing_tcs))
    
#print("IDF-Threshold Value = " + str(idf_values))

In [3]:
# Loading Coverage File

coverage_information_main = pd.read_csv(input_coverage_file_path)

# Display top 5 rows o coverage file.
#coverage_information_main.head()

In [4]:
# Processing of Coverage Data

# 1. Sort By Test Case Classes - from Coverage Data

names_of_test_case_classes = coverage_information_main['Test Case'].unique()

#print("Number of Test Classes = " + str(names_of_test_case_classes.size) + "\n")
#print("Names of Test Classes = " + str(names_of_test_case_classes) + "\n")


# 2. Sort By Code Classes - from Coverage Data

names_of_code_classes = coverage_information_main['CLASS'].unique()

#print("Number of Code Classes = " + str(names_of_code_classes.size) + '\n')
#print("Names of Code Classes = " + str(names_of_code_classes)+ '\n')

group_by_test_classes  = coverage_information_main.groupby('Test Case')
group_by_code_classes  = coverage_information_main.groupby('CLASS')


no_of_tcs_impacted_by_each_class = group_by_test_classes.count()[['CLASS']]
no_of_classes_impacted_by_each_test_case = group_by_code_classes.count()[['Test Case']]


no_of_classes_impacted_by_each_test_case = no_of_classes_impacted_by_each_test_case.reset_index()
no_of_tcs_impacted_by_each_class = no_of_tcs_impacted_by_each_class.reset_index()


In [5]:
# Calculate size of coverage

lines_coverage_size = coverage_information_main.groupby('Test Case')['LINE_COVERED'].sum()
branch_coverage_size = coverage_information_main.groupby('Test Case')['BRANCH_COVERED'].sum()
method_coverage_size = coverage_information_main.groupby('Test Case')['METHOD_COVERED'].sum()

dict_sizes = { 'MethodCoverage': method_coverage_size, 'BranchCoverage': branch_coverage_size , 'LineCoverage': lines_coverage_size }

# Combined DF Coverage Size
df_sizes = pd.DataFrame(dict_sizes)


In [6]:
# Calculation of Y-Value

y_value_column_names = df_y_values.columns
df_y_values = df_y_values.fillna("-")

sample_dict = {}


names_of_test_case_classes.sort

for tcs in names_of_test_case_classes:
    for i in range(len(y_value_column_names)):
        tcs_tmp = df_y_values[y_value_column_names[i]].loc[df_y_values[y_value_column_names[i]].values != '-']
        if tcs in df_y_values[y_value_column_names[i]].values:
            if tcs not in sample_dict:
                sample_dict[tcs] = list()
            no_of_tcs_finding_defect = len(tcs_tmp)
            value_of_y_tc = 1/(np.log10(no_of_tcs_finding_defect)+1)
            sample_dict[tcs].append(value_of_y_tc)


pp = pd.DataFrame.from_dict(sample_dict, orient='index')

pp["Y-value"] = pp.sum(axis=1)
pp = pp.reset_index()

pp=pp.rename(columns = {'index':'Test Case'})
#pp

In [15]:
# Calculation of TF-IDF Scores for Each Code Class


no_of_classes_impacted_by_each_test_case['totalTestsDivideCoveredTest'] = names_of_test_case_classes.size/(no_of_classes_impacted_by_each_test_case['Test Case'])

no_of_classes_impacted_by_each_test_case['TF-value'] = 1

no_of_classes_impacted_by_each_test_case['IDF'] = np.log10(no_of_classes_impacted_by_each_test_case['totalTestsDivideCoveredTest']) +1 


temp_df_idf_for_threshold = no_of_classes_impacted_by_each_test_case.copy()
coverage_information_main.reset_index()
no_of_classes_impacted_by_each_test_case
# Calculation of Similarity Scores - based on the threshold values 
ia = 0
for threshold_val in idf_values:
    df_threshold_idf_condition = pd.DataFrame()
    tf_id_value_df = pd.DataFrame()
    mapping_between_tc_and_code_class = pd.DataFrame()
    merged_inner = pd.DataFrame()
    similarity_score_df = pd.DataFrame()
    df_training_set = pd.DataFrame()
    similarity_score_df = pd.DataFrame()
    df_threshold_idf_condition = temp_df_idf_for_threshold.loc[temp_df_idf_for_threshold['Test Case'] < threshold_val]
    
    # Calculation of Similarity Score 
    
    # A data frame containing IDF and TF Values 
    tf_id_value_df = df_threshold_idf_condition[['CLASS', 'IDF', 'TF-value']]

    # Map each test case vs the code class it covers
    mapping_between_tc_and_code_class = coverage_information_main[['CLASS', 'Test Case']]

    # Merge (inner merge) two data frames (TF/IDF of classes dataframe and mapping between classes and test cases)
    merged_inner = pd.merge(left=mapping_between_tc_and_code_class, right=tf_id_value_df, left_on='CLASS', right_on='CLASS')

    # Data Frame Calculaing Similarity Score of Each Test Case
    similarity_score_df.empty
    similarity_score_df = merged_inner.groupby('Test Case')['IDF'].sum().to_frame().reset_index()
    similarity_score_df.columns = ['Test Case', 'Similarity Score']
    
    df_training_set = pd.DataFrame()
    df_training_set = pd.merge(left=df_sizes, right=similarity_score_df, on='Test Case', how='outer')
    df_training_set= df_training_set.fillna(0)
    #print(" IDF Value = " + str(threshold_val))
    #print(df_training_set)
    
    
    # Add Y-values to the features
    final_set = pd.merge(left=df_training_set, right=pp[['Test Case','Y-value']], on='Test Case', how='outer')

    final_set = final_set.fillna(0)
    #print(final_set)
    linear_regress = LinearRegression()
    # Fit the multiple Linear regression model
    # x is ['Similarity Ratio', 'Coverage Size']

    #print(type(final_set[['MethodCoverage', 'Similarity Score']]))
    #print(type(final_set['Y-value']))
    linear_regress.fit(final_set[['MethodCoverage', 'Similarity Score']], final_set['Y-value'])

    print("\n\nFOR IDF Threshold = "  + str(threshold_val) + "  Below are Theta Values ")
    
    #Calculating Theta 0, Theta 1, Theta 2.

    print("Theta 0 = " + str(linear_regress.intercept_) + "\nTheta 1 = " + str(linear_regress.coef_[0]) + "\nTheta 2 = " + str(linear_regress.coef_[1]) )



FOR IDF Threshold = 2  Below are Theta Values 
Theta 0 = -0.34526028553407373
Theta 1 = 0.010163421783841851
Theta 2 = -0.012274522769867556


FOR IDF Threshold = 3  Below are Theta Values 
Theta 0 = -0.34526028553407373
Theta 1 = 0.010163421783841851
Theta 2 = -0.012274522769867556


FOR IDF Threshold = 4  Below are Theta Values 
Theta 0 = -0.33952287582074503
Theta 1 = 0.00986976078523841
Theta 2 = 0.011566421795649944


FOR IDF Threshold = 6  Below are Theta Values 
Theta 0 = -0.29961701959211917
Theta 1 = 0.007365401119177318
Theta 2 = 0.045818553187739045


FOR IDF Threshold = 8  Below are Theta Values 
Theta 0 = -0.29961701959211917
Theta 1 = 0.007365401119177318
Theta 2 = 0.045818553187739045


FOR IDF Threshold = 9  Below are Theta Values 
Theta 0 = -0.29961701959211917
Theta 1 = 0.007365401119177318
Theta 2 = 0.045818553187739045


FOR IDF Threshold = 10  Below are Theta Values 
Theta 0 = -0.297774169692261
Theta 1 = 0.007301034296103368
Theta 2 = 0.03875919318089372


FOR I

In [8]:
# BACKUP IMPLEMENTAION - DO NOT TOUCH
# Calculation of Similarity Score Backup 
# A data frame containing IDF and TF Values 
#tf_id_value_df = no_of_classes_impacted_by_each_test_case[['CLASS', 'IDF', 'TF-value','Test Case']]

#tf_id_value_df
# Calculation of Y-values


# Map each test case vs the code class it covers
#coverage_information_main.reset_index()
#mapping_between_tc_and_code_class = coverage_information_main[['CLASS', 'Test Case']]

#mapping_between_tc_and_code_class.loc[mapping_between_tc_and_code_class['Test Case'] == 'ApplicationTest']


# Merge (inner merge) two data frames (TF/IDF of classes dataframe and mapping between classes and test cases)
#merged_inner = pd.merge(left=mapping_between_tc_and_code_class, right=tf_id_value_df, left_on='CLASS', right_on='CLASS')
#aa = merged_inner.loc[merged_inner['Test Case_y'] < 14]
#aa.loc[aa['Test Case_x'] == 'ApplicationTest']
# Data Frame Calculaing Similarity Score of Each Test Case
#similarity_score_df = merged_inner.groupby('Test Case')['IDF'].sum().to_frame().reset_index()
#similarity_score_df.columns = ['Test case', 'Similarity Score']
#similarity_score_df


#OBSOLETE: Calculation of Y-Value

#df_y_values = pd.read_csv('y-values.csv')
#y_value_column_names = df_y_values.columns
#df_y_values = df_y_values.fillna("-")

#sample_dict = {}
#names_of_test_case_classes.sort

#for tcs in names_of_test_case_classes:
    #for i in range(len(y_value_column_names)):
       # tcs_tmp = df_y_values[y_value_column_names[i]].loc[df_y_values[y_value_column_names[i]].values != '-']
      #  if tcs in df_y_values[y_value_column_names[i]].values:
     #       if tcs not in sample_dict:
    #            sample_dict[tcs] = list()
   #         no_of_tcs_finding_defect = len(tcs_tmp)
  #          value_of_y_tc = 1/(np.log10(no_of_tcs_finding_defect)+1)
 #           sample_dict[tcs].append(value_of_y_tc)
#

#pp = pd.DataFrame.from_dict(sample_dict, orient='index')

#pp["Y-value"] = pp.sum(axis=1)

# Linear Regression

#linear_regress = LinearRegression()
# Fit the multiple Linear regression model
# x is ['Similarity Ratio', 'Coverage Size']
#linear_regress.fit(x,y)

# Three co-efficients 
#linear_regress.coef_